In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

%cd gdrive/My Drive/University/MAIS202/Final Project

Mounted at /content/gdrive
/content/gdrive/My Drive/University/MAIS202/Final Project


In [0]:
import os
import re

from sklearn.model_selection import train_test_split

In [0]:
max_word_len = 20

In [0]:
path_data = 'data'

path_cmu_dict = os.path.join(path_data, 'cmudict.txt')

paths = {'src':{}, 'tgt':{}}

for data_type in paths:
    for set_type in ('train', 'val', 'test'):
        paths[data_type][set_type] = os.path.join(path_data, '{}-{}.txt'.format(data_type, set_type))

paths

# path_src_train = os.path.join(path_data, 'src-train.txt')
# path_tgt_train = os.path.join(path_data, 'tgt-train.txt')
# path_src_val = os.path.join(path_data, 'src-val.txt')
# path_tgt_val = os.path.join(path_data, 'tgt-val.txt')
# path_src_test = os.path.join(path_data, 'src-test.txt')
# path_tgt_test = os.path.join(path_data, 'tgt-test.txt')

{'src': {'test': 'data/src-test.txt',
  'train': 'data/src-train.txt',
  'val': 'data/src-val.txt'},
 'tgt': {'test': 'data/tgt-test.txt',
  'train': 'data/tgt-train.txt',
  'val': 'data/tgt-val.txt'}}

In [0]:
def is_valid(word):
    return re.match('[A-Z]*$', word) and len(word) <= max_word_len

def process(word):
    # match = re.search('[A-Z]*', word)
    # return match.group()
    return word.replace("", " ")[1: -1]

def remove_last(word):
    return re.sub('(\(\d+\))?$', '', word)

def write_list_to_file(l, path_file):
    with open(path_file, 'w') as file:
        for item in l[:-1]:
            file.write(item + '\n')
        file.write(l[-1])

In [0]:
with open(path_cmu_dict, encoding='ISO-8859-1') as file_dict:
    
    # i = 0

    clean_dict = {}
    words_list = []
    pronounciations_list = []

    for line in file_dict:
    
        # skip comments
        if line[:3] == ';;;':
            continue
        
        # split line into word and pronounciation
        word, pronounciation = line.strip().split('  ')
        
        # check if word is valid
        if is_valid(word):
            
            # print('{}: {}'.format(word, pronounciation))

            word_processed = process(word)

            if word_processed not in clean_dict:
                clean_dict[word_processed] = pronounciation
                words_list.append(word_processed)
                pronounciations_list.append(pronounciation)

        #     i += 1
        # if i == 5:
        #     break

# print(clean_dict)
                    

In [0]:
print({k: clean_dict[k] for k in list(clean_dict)[:20]})
print(words_list[:20])
print(pronounciations_list[:20])

{'A': 'AH0', 'A A': 'EY2 EY1', 'A A A': 'T R IH2 P AH0 L EY1', 'A A B E R G': 'AA1 B ER0 G', 'A A C H E N': 'AA1 K AH0 N', 'A A C H E N E R': 'AA1 K AH0 N ER0', 'A A H': 'AA1', 'A A K E R': 'AA1 K ER0', 'A A L I Y A H': 'AA2 L IY1 AA2', 'A A L S E T H': 'AA1 L S EH0 TH', 'A A M O D T': 'AA1 M AH0 T', 'A A N C O R': 'AA1 N K AO2 R', 'A A R D E M A': 'AA0 R D EH1 M AH0', 'A A R D V A R K': 'AA1 R D V AA2 R K', 'A A R D V A R K S': 'AA1 R D V AA2 R K S', 'A A R G H': 'AA1 R G', 'A A R O N': 'EH1 R AH0 N', 'A A R O N S': 'EH1 R AH0 N Z', 'A A R O N S O N': 'EH1 R AH0 N S AH0 N', 'A A R T I': 'AA1 R T IY2'}
['A', 'A A', 'A A A', 'A A B E R G', 'A A C H E N', 'A A C H E N E R', 'A A H', 'A A K E R', 'A A L I Y A H', 'A A L S E T H', 'A A M O D T', 'A A N C O R', 'A A R D E M A', 'A A R D V A R K', 'A A R D V A R K S', 'A A R G H', 'A A R O N', 'A A R O N S', 'A A R O N S O N', 'A A R T I']
['AH0', 'EY2 EY1', 'T R IH2 P AH0 L EY1', 'AA1 B ER0 G', 'AA1 K AH0 N', 'AA1 K AH0 N ER0', 'AA1', 'AA1 

In [0]:
words = {}
pronounciations = {}

words['train'], words['test'], pronounciations['train'], pronounciations['test'] = train_test_split(
    words_list, pronounciations_list, test_size=0.2, random_state=3791)

words['train'], words['val'], pronounciations['train'], pronounciations['val'] = train_test_split(
    words['train'], pronounciations['train'], test_size=0.125, random_state=3791)

print(len(words['train']), len(words['test']), len(words['val']), len(
    pronounciations['train']), len(pronounciations['test']), len(pronounciations['val']))

81551 23301 11651 81551 23301 11651


In [0]:
for set_type in ('train', 'test', 'val'):
    write_list_to_file(words[set_type], paths['src'][set_type])
    write_list_to_file(pronounciations[set_type], paths['tgt'][set_type])

In [0]:
chars = set()
phonemes = set()

for word in words_list:
    for char in word.split():
        chars.add(char)

for pronounciation in pronounciations_list:
    for phoneme in pronounciation.split():
        phonemes.add(phoneme)

print(len(chars))
print(chars)

print(len(phonemes))
print(phonemes)

write_list_to_file(list(chars), os.path.join(path_data, 'src-vocab.txt'))
write_list_to_file(list(phonemes), os.path.join(path_data, 'tgt-vocab.txt'))

26
{'P', 'F', 'R', 'D', 'Z', 'E', 'C', 'J', 'X', 'N', 'O', 'A', 'B', 'I', 'Q', 'W', 'Y', 'V', 'K', 'L', 'S', 'H', 'T', 'M', 'U', 'G'}
69
{'SH', 'AY1', 'IY0', 'OW0', 'EH1', 'IH1', 'P', 'AW0', 'IH2', 'OW2', 'IH0', 'F', 'R', 'D', 'ER0', 'AH2', 'ER2', 'Z', 'OW1', 'UH1', 'AW1', 'EY2', 'OY2', 'N', 'EY0', 'AY2', 'EH0', 'B', 'AE0', 'UW0', 'EH2', 'AE1', 'UW2', 'UW1', 'AO0', 'W', 'IY2', 'Y', 'AO1', 'AY0', 'AW2', 'CH', 'V', 'UH2', 'HH', 'UH0', 'AH1', 'IY1', 'AE2', 'JH', 'ZH', 'ER1', 'K', 'AH0', 'L', 'OY0', 'AO2', 'S', 'DH', 'EY1', 'AA0', 'OY1', 'TH', 'AA1', 'NG', 'T', 'AA2', 'M', 'G'}
